In [1]:
#!pip install plotly pandas numpy scipy yfinance scipy.stats scipy.signal plotly.express

#!python.exe -m pip install --upgrade pip


In [2]:
# Import libraries
import pandas as pd
import numpy as np
import scipy.stats as stats
import scipy.signal as signal
import plotly.express as px
import yfinance as yf



In [3]:

# Define parameters
symbol = "ES=F" # S&P 500-mini Futures (/ES) index symbol
per = "4y" # 4 years period
inter = "1d" # 1 day interval
kde_factor = 0.01 # KDE smoothness factor
prominence = 0.25 # Minimum peak prominence
width = 100 # Maximum peak width in points


In [4]:

# Fetch data from yahoo finance
spx = yf.Ticker(symbol)
df = spx.history(period=per, interval=inter)


In [5]:

df = df.loc[df.index > df.index.max() - pd.DateOffset(years=2)] # Filter for last 2 years
df.drop(columns=["Dividends", "Stock Splits","Open", "High", "Low"], inplace=True) # Drop dividends and stock splits columns


In [6]:

# Plot volume profile
fig = px.histogram(df, x="Close", y="Volume", histfunc="sum", nbins=1200)
fig.show()

# Fit KDE to volume profile
kde = stats.gaussian_kde(df["Close"], weights=df["Volume"], bw_method=kde_factor)
x = np.linspace(df["Close"].min(), df["Close"].max(), 1200)
y = kde(x)

# Plot KDE curve and normalized histogram
fig = px.line(x=x, y=y, labels={"x": "Price", "y": "Density"})
fig.add_histogram(x=df["Close"], y=df["Volume"], histfunc="sum", nbinsx=1200, histnorm="probability density", bingroup=None)
fig.show()


In [7]:

# Find peaks in KDE curve
peaks, properties = signal.find_peaks(y, prominence=prominence*(y.max()-y.min()))

# Plot peaks and their properties
fig = px.line(x=x, y=y, labels={"x": "Price", "y": "Density"})
fig.add_scatter(x=x[peaks], y=y[peaks], mode="markers", marker_color="red")
for i in range(len(peaks)):
    fig.add_vline(x=x[int(properties["left_bases"][i])], line_dash="dash", line_color="green")
    fig.add_vline(x=x[int(properties["right_bases"][i])], line_dash="dash", line_color="green")
    #fig.add_hline(y=properties["prominences"][i], line_dash="dash", line_color="blue")
fig.show()


In [8]:

# Print peak prices and densities
print("Peak prices and densities:")
for i in range(len(peaks)):
    print(f"{x[peaks[i]]:.2f}, {y[peaks[i]]:.6f}")


Peak prices and densities:
3599.49, 0.001103
3654.42, 0.000826
3708.35, 0.000806
3732.32, 0.001032
3767.28, 0.000970
3805.23, 0.001327
3871.15, 0.001976
3916.09, 0.001798
3969.02, 0.002016
3988.00, 0.002080
4001.98, 0.002635
4032.94, 0.001631
4075.89, 0.001585
4154.79, 0.002908
4171.77, 0.001724
4210.72, 0.001410
4299.61, 0.001640
4341.56, 0.001711
4389.50, 0.001932
4409.47, 0.001696
4506.35, 0.001685
4537.31, 0.001791
4566.28, 0.001386
4596.24, 0.001461
4688.12, 0.001342
